# Outils de création d'une carte interactive par catégories

Fichier nécessaire: un fichier en format .txt contenant la liste des noms de lieux à recenser sur la carte.

Les outils de recherches d'informations permettent de récupérer les emplacements de chaque lieu (coordonnées GPS), le type de lieu dans lequel il est catégorisé ainsi qu'une brève description du lieu. Un fichier sous la forme d'un tableau lisible avec Excel est créé avec les informations récoltées.

La récupération des types de lieux permet de trier les ieux par catégories et de créer une carte interactive en fonction du classement créé. 

La carte est enregistrée en format html.

Il est possible à la fin d'ajouter, à la fin du notebook, son propre fond de carte.

**Avant de lancer le notebook**:

Créer un dossier sur son Google Drive intitulé "Creation-de-cartes".
Placer dans ce dossier ce notebook, ainsi que le fichier .txt contenant la liste des lieux.


## Connexion à son compte GoogleDrive:

- Lancer la cellule
- Cliquer sur le lien généré en dessous
Copier le mot de passe dans la nouvelle fenêtre ouverte où l'accès au compte est requis et le coller en dessous de la cellule.

In [ ]:
!pip install geopy
!pip install --upgrade wptools
from google.colab import drive
import folium
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import csv
import wptools
import re
import itertools
import urllib.parse, urllib.request
from collections import Counter
import matplotlib.pyplot as plt
from pandas.plotting import table

drive.mount('/content/drive/')
%cd /content/drive/My Drive/Creation-de-carte/

## Récupération des identifiants WIKIDATA des lieux

- Indiquer le chemin du fichier .txt avec la liste des lieux (fin de la cellule)
+ possibilité de changer le nom du fichier de sortie (fichier .txt avec liste des identifiants)
- Changer la langue si nécessaire (par défaut: français)
- Si la détéction n'est pas assez précis, descendre "thresold" à 0.8 (moins de noms vont être identifiés), dans la cas contraire, monter le seuil à 1. Par défaut, il est placé à une valeur moyenne de 0.9

In [ ]:
def CallWikifier(text, lang="fr", threshold=1): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
    with open('link-lieux2.txt', 'w') as outfile:
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          itemid = annotation["wikiDataItemId"] 
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)

with open ("test-it-lieux.txt", "r") as myfile:
    liste=myfile.read()
CallWikifier(text=liste, lang="fr")

## Récupération des informations de chaque lieu reconnu et lié à wikidata:

- Lancer la cellule

Toutes les informations (le type de lieu, les coordonnées GPS (latitude et longitude) ainsi qu'une brève description du lieu) sont enregistrées et exportées sous la forme d'un fichier de données JSON ainsi que sous la forme d'un tableau dans un fichier Excel dans le Google Drive. Il est possbile de changer le nom du fichier xlsx créé à la toute dernière ligne de la cellule: ici, il se nomme "informations_lieux.xlsx"

In [ ]:
%cd /content/drive/My Drive/Creation-de-carte/

donnees ={}
nom = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []
# Utilisation du document créé précédement avec les identifiants wikidata:
with open("/content/drive/My Drive/Creation-de-carte/test-it-lieux (1).txt", "r") as a_file:
      with open("donnees_lieux.json", "w") as fichier : 
        for line in a_file:
            for word in line.split():
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations:
              try:
                infobox=page.data['infobox']
                nom = infobox.get("nom")
                type_lieu=infobox.get("type")
                if type_lieu is None: 
                  type_lieu=page.data['what']
                latitude = infobox.get("latitude")
                longitude = infobox.get("longitude")
                description = page.data["description"]
              except AttributeError as at:
                print("Ce n'est pas un lieu!")
# Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux:
              donnees[nom] = {"Type de lieu" : type_lieu,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description}
      
        json.dump(donnees, fichier, ensure_ascii=False)


# Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel('informations_lieux.xlsx')

##Outils de calculs statistiques:

Calcul des types de lieux détectés sur wikidata ainsi que la proportion de chaque.

- Lancer la cellule.

- Les résultats s'affichent en dessous : le nombre de lieux par "type" ainsi que la proportion de chaque catégorie.

In [8]:
# Récupération et création d'une liste avec catégories de lieux
keys = donnees[nom]["Type de lieu"]
liste_des_types = []
fichier_json = "/content/drive/My Drive/Creation-de-carte/donnees_lieux.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)


def proportion_types(liste):
  c = Counter(liste)
  print("Voici le nombre de lieux, triés par chaque type:", "\n", c) ## Affiche le nombre de lieux de chaque type
  prop = [(i, c[i] / len(liste) * 100.0) for i in c]
  print("Voici la proportion de chaque type de lieux:", "\n", prop) ## Affiche la proportion de chaque type de lieux
  
proportion_types(liste_des_types)



Voici le nombre de lieux, triés par chaque type: 
 Counter({'place': 3, 'rue': 2, 'lieu-dit': 1, '[[Tour autoportante]]': 1, 'station de métro': 1, '[[École consulaire]]': 1, 'parc zoologique': 1, 'Château fort, arsenal et prison': 1, "jardin à l'anglaise": 1, 'commune de France': 1, 'Lac artificiel': 1})
Voici la proportion de chaque type de lieux: 
 [('place', 21.428571428571427), ('lieu-dit', 7.142857142857142), ('[[Tour autoportante]]', 7.142857142857142), ('station de métro', 7.142857142857142), ('rue', 14.285714285714285), ('[[École consulaire]]', 7.142857142857142), ('parc zoologique', 7.142857142857142), ('Château fort, arsenal et prison', 7.142857142857142), ("jardin à l'anglaise", 7.142857142857142), ('commune de France', 7.142857142857142), ('Lac artificiel', 7.142857142857142)]


## Création de la carte avec outils de sélection par types de lieux :

- Lancer la cellule. 

- La catégorisation des lieux faite par wikidata est générée automatiquement pour permettre la sélection de certains types de lieux en particulier.

- La carte est enregistrée automatiquement en format .html dans le dossier courant dans le GoogleDrive. Il est possible de changer le nom de la carte crée (ici, elle s'intitule "carte_creee.html"). 
Il faut télécharger ensuite la carte du Google Drive sur son ordinateur pour pouvoir l'ouvrir dans un navigateur.

In [11]:
# Faire liste des types de lieux
keys = donnees[nom]["Type de lieu"]
liste_des_types = []
fichier_json = "/content/drive/My Drive/Creation-de-carte/donnees_lieux.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)


items = Counter(liste_des_types).keys() 
nbr_types_lieux = len(items)

### Affilier un numéro de groupe pour chaque catégorie
groupe = []
d = {ni: indi for indi, ni in enumerate(set(liste_des_types))}
numbers = [d[ni] for ni in liste_des_types]

for k, v in zip(contenu_json, numbers):
  groupe.append(v)
  for element in groupe:
    contenu_json[k]["Groupe"] = element

### Création du fonds de carte
map = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
fg = folium.FeatureGroup(name="Tous")
map.add_child(fg)

## Création des différentes catégories pour la carte
def creating_categories(nombre_categories, liste_noms_categories):
  g = []
  for k in contenu_json: 
    lat = contenu_json[k]["Latitude"] 
    longi = contenu_json[k]["Longitude"]
    for i, j in zip(range(nombre_categories), liste_noms_categories): 
      g=plugins.FeatureGroupSubGroup(fg, str(j)) 
      try:
        if str(j) == str(donnees[k]["Type de lieu"]):  
          try:
            folium.Marker([float(lat), float(longi)]).add_to(g) 
            map.add_child(g)  
          except TypeError as Te:
            pass
      except KeyError as kerr:
        pass

creating_categories(nbr_types_lieux, liste_des_types)

folium.LayerControl(collapsed=False).add_to(map)

map.save("carte_categories_creee.html")
map


## Créer et ajouter son fond de carte

Si l'on veut ajouter son PROPRE fond de carte personnalisé:

- Création d’un compte sur : https://mapwarper.net/

- Téléchargement du fond de carte (IIIF recommandé, avec la meilleure résolution possible)

- Sur Map Warper: dans l’onglet « Upload Map » il est possible de charger le fond de carte que l’on veut utiliser, et indiquer toutes les métadonnées nécessaires.

- Dans l’onglet "Rectify": il faut indiquer le plus de points identiques sur les deux cartes pour permettre la superposition du fond de carte chargé sur des données géolocalisées. Cliquer sur un point sur chaque carte, puis cliquer sur « add Control point » en dessous des cartes. Répéter l’opération le plus de fois possible pour obtenir un bon résultat. Dans l'encadré en dessous des deux fonds de carte, cliquer sur "Warp image": le fond s'ajoute. Si le résultat n'est pas satisfaisant, il possbile d'ajouter d'autres repères pour améliorer la qualité du nouveau fond de carte créé. 

- Se rendre dans l'onglet "Export". Dans "Map services", copier le lien entier se situant après "Tiles (Google/OSM scheme):", débutant par http et se terminant par .png

- Coller ensuite le lien à la troisième ligne de la cellule suivante, à la place du lien existant, après "tiles = " en conservant bien les guillemets.

In [12]:
### Création du fonds de carte
map = folium.Map(location=[48.856614, 2.3522219], zoom_start=12,
                 tiles="https://mapwarper.net/maps/tile/54525/{z}/{x}/{y}.png", attr="None")
fg = folium.FeatureGroup(name="Tous")
map.add_child(fg)

# Faire liste des types de lieux
keys = donnees[nom]["Type de lieu"]
liste_des_types = []
fichier_json = "/content/drive/My Drive/Creation-de-carte/donnees_lieux.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)


items = Counter(liste_des_types).keys() 
nbr_types_lieux = len(items)

### Affilier un numéro de groupe pour chaque catégorie
groupe = []
d = {ni: indi for indi, ni in enumerate(set(liste_des_types))}
numbers = [d[ni] for ni in liste_des_types]

for k, v in zip(contenu_json, numbers):
  groupe.append(v)
  for element in groupe:
    contenu_json[k]["Groupe"] = element

## Création des différentes catégories pour la carte
def creating_categories(nombre_categories, liste_noms_categories):
  g = []
  for k in contenu_json: ### boucle: pour chasue lieu dontenu dans "donnees"
    lat = contenu_json[k]["Latitude"] 
    longi = contenu_json[k]["Longitude"]
    for i, j in zip(range(nombre_categories), liste_noms_categories): 
      g=plugins.FeatureGroupSubGroup(fg, str(j)) 
      try:
        if str(j) == str(donnees[k]["Type de lieu"]):  
          try:
            folium.Marker([float(lat), float(longi)]).add_to(g) 
            map.add_child(g)  
          except TypeError as Te:
            pass
      except KeyError as kerr:
        pass

creating_categories(nbr_types_lieux, liste_des_types)

folium.LayerControl(collapsed=False).add_to(map)

map.save("carte_categories_creee.html")
map
